# Dataframe
- 데이터프레임은 RDBM의 테이블에서 column이름으로 구성된 immutable한 분산데이터 컬렉션임. Apache Spark 1.3에서 Schema-RDD가 데이터프레임이라는 이름으로 변경됨.
- 분산된 데이터 컬렉션에 구조체를 씌움으로써 SparkSQL로 구조적 데이터를 쿼리하거나 lambda대신에 표현함수(expression method)를 사용할 수 있음.
- 데이터를 구조적으로 바꿈으로써 스파크 엔진(카탈리스트 옵티마이저)의 스파크 쿼리 성능을 크게 향상시켰음.
- RDD에서 쿼리를 파이썬에서 실행하는 것은 자바 JVM과 Py4J 사이의 커뮤티케이션 오버헤드 때문에 크게 느린점을 극복하였음

### 데이터프레임 생성하기
- SparkSession()을 이용하여 데이터를 import하는 방식으로 데이터프레임을 생성할 것

### JSON데이터 생성하기

In [1]:
stringJSONRDD = sc.parallelize([
    # id, 이름, 나이, 눈 색깔
    """{"id": "123", "name": "Kattie", "age":19, "eyeColor" : "brown"}""",
    """{"id": "234", "name": "Michael", "age": 22, "eyeColor": "green"}""", 
    """{
        "id": "345",
        "name": "Simone",
        "age": 23,
        "eyeColor": "blue"
  }"""
])

sc.parallelize는 아직 실행되지 않는 트랜스포메이션이라는 것을 기억!!!

### 데이터프레임 생성

In [2]:
swimmersJSON = spark.read.json(stringJSONRDD)
swimmersJSON.take(3)

[Row(age=19, eyeColor='brown', id='123', name='Kattie'),
 Row(age=22, eyeColor='green', id='234', name='Michael'),
 Row(age=23, eyeColor='blue', id='345', name='Simone')]

stringJSONRDD는 spark.read.json을 사용해서 RDD를 데이터프레임으로 변환할 때 실행되는 트랜스포메이션임. 스파크 잡은 spark.read.json을 포함하는 셀을 실행하기 이전까지 실행되지 않는다!!!

parallelize, map, mapPartitions가 모두 RDD 트랜스포메이션이라는 것은 매우 중요한 점임. 데이터프레임 작업 내에 감싸여서 spark.read.json은 RDD 트랜스포메이션일 뿐만 아니라 RDD를 데이터프레임으로 바꾸는 액션이기도 함(매우 중요함). 데이터프레임 작업을 실행하고 있더라도, 디버깅 목적으로 스파크 UI내에서 RDD작업이 가능함을 알고 있어야 하기 때문임

### 임시 테이블 생성

In [3]:
# 임시테이블 혹은 테이블을 생성하지 않으면 SparkSQL은 물론이고 SQL도 사용할 수 없음
swimmersJSON.createOrReplaceTempView("swimmersJSON")

임시테이블을 생성하는 것은 데이터프레임 트랜스포메이션이고 데이터프레임 액션이 실행되기 전까지 실행되지 않는다는 것을 잊으면 안됨.

### 데이터프레임 API 쿼리

In [4]:
swimmersJSON.show()

+---+--------+---+-------+
|age|eyeColor| id|   name|
+---+--------+---+-------+
| 19|   brown|123| Kattie|
| 22|   green|234|Michael|
| 23|    blue|345| Simone|
+---+--------+---+-------+



### SQL쿼리

In [5]:
spark.sql("select * from swimmersJSON").collect()

[Row(age=19, eyeColor='brown', id='123', name='Kattie'),
 Row(age=22, eyeColor='green', id='234', name='Michael'),
 Row(age=23, eyeColor='blue', id='345', name='Simone')]

모든 데이터를 행 Row()객체로 변환하는 collect()함수를 사용했음. collect()함수나 show()함수는 데이터프레임과 SQL쿼리에 대해 사용할 수 있음. 
 - collect()함수는 데이터프레임의 모든 행을 리턴하고 실행 노드에서 드라이버 노드로 이동하기 때문에 작은 데이터프레임에 사용하는 것이 좋음. 
 - 그렇기 때문에 n을 사용해 리턴되는 행의 갯수를 제한하는 **take**나 **show**를 사용하는 것이 좋음

In [6]:
spark.sql("select * from swimmersJSON").show(3)

+---+--------+---+-------+
|age|eyeColor| id|   name|
+---+--------+---+-------+
| 19|   brown|123| Kattie|
| 22|   green|234|Michael|
| 23|    blue|345| Simone|
+---+--------+---+-------+



In [7]:
spark.sql('select * from swimmersJSON').take(3)

[Row(age=19, eyeColor='brown', id='123', name='Kattie'),
 Row(age=22, eyeColor='green', id='234', name='Michael'),
 Row(age=23, eyeColor='blue', id='345', name='Simone')]

databricks notebook을 사용하면 아래와 같은 명령어를 사용할 수 있음
버전업? 지원X
```python
%load_ext sql select * from swimmersJSON
```

## 데이터 프레임과 RDD연동
기존에 있는 RDD를 데이터프레임(Dataset[T])으로 변경하는 방법은 2가지
- 리플렉션(reflection)을 사용해 스키마를 추측하는 것
- 스키마를 직접적으로 코드상에 명싱하는 방법

리플렉션을 사용하면 더욱 자세한 코드를 작성할 수 있는 반면에, 스키마를 명시하는 방법은 열과 데이터 타입이 런타임에 드러날 때 데이터프레임의 구조를 만들도록해야 함

### 1. reflection을 이용한 스키마 추측하기
데이터프레임을 빌드하고 쿼리를 수행하는 과정에서 이 데이터프레임에 대한 스키마는 자동으로 정의됨. 최초의 행 객체는 key/value 쌍 리스트가 행 클래스의 \*\*kwargs로 전달되어 구성됨. 그 후, SparkSQL은 이 행 객체의 RDD를 데이터프레임으로 변경함. 키는 칼럼이고 데이터 타입은 데이터 샘플링을 통해 추측됨

In [8]:
swimmersJSON.printSchema()

root
 |-- age: long (nullable = true)
 |-- eyeColor: string (nullable = true)
 |-- id: string (nullable = true)
 |-- name: string (nullable = true)



id를 string이 아니라 long타입으로 변경하는 작업을 수행해보자

### 2. 프로그래밍하는 것처럼 스키마 명시
- SparkSQL 데이터 타입을 불러와서 사용
- StructField(name, dataType, nullable)
    - name : 필드의 이름
    - dataType : 필드의 데이터타입
    - nullable : 필드가 null이 될 수 있는지 명시

In [9]:
from pyspark.sql.types import *

# 콤마로 분리된 데이터를 생성
stringCSVRDD = sc.parallelize([
    (123, 'Kattie', 19, 'brown'),
    (234, 'Michael', 22, 'green'),
    (345, 'Simone', 23, 'blue')
])

# StructType과 StructField를 사용하여 정의
schema = StructType([
    StructField("id", LongType(), True),
    StructField("name", StringType(), True),
    StructField("age", LongType(), True),
    StructField("eyeColor", StringType(), True)
])

In [10]:
swimmers = spark.createDataFrame(stringCSVRDD, schema)
swimmers.createOrReplaceTempView("swimmers")

In [11]:
swimmers.printSchema()

root
 |-- id: long (nullable = true)
 |-- name: string (nullable = true)
 |-- age: long (nullable = true)
 |-- eyeColor: string (nullable = true)



### 데이터프레임 API로 쿼리하기
- 행의 갯수
- 필터문 실행

In [12]:
swimmers.count()

3

In [13]:
# 방법
swimmers.select("id", "age").filter("age = 22").show()

+---+---+
| id|age|
+---+---+
|234| 22|
+---+---+



In [14]:
display(swimmers)

DataFrame[id: bigint, name: string, age: bigint, eyeColor: string]

In [15]:
swimmers.select(swimmers.id, swimmers.age).filter(swimmers.age == 22).show()

+---+---+
| id|age|
+---+---+
|234| 22|
+---+---+



눈 색깔이 b로 시작하는 수영 선수의 이름을 얻어보자

In [16]:
import time
start_time = time.time()

# 트랜스포메이션 액션을 사용하는 것이 좋음
swimmers.select("name", "eyeColor").filter("eyeColor like 'b%'").show()
total_time = time.time() - start_time
print(total_time)

+------+--------+
|  name|eyeColor|
+------+--------+
|Kattie|   brown|
|Simone|    blue|
+------+--------+

0.16889357566833496


### SQL로 쿼리하기

In [17]:
spark.sql("select count(1) from swimmers").show()

+--------+
|count(1)|
+--------+
|       3|
+--------+



### 필터문을 where절을 사용해 실행하기

In [18]:
start_time = time.time()
spark.sql("select name, eyeColor from swimmers where eyeColor like 'b%'").show()
total_time = time.time() - start_time
print(total_time)

+------+--------+
|  name|eyeColor|
+------+--------+
|Kattie|   brown|
|Simone|    blue|
+------+--------+

0.15664029121398926


## 데이터프레임 시나리오 : 비행 기록 성능

In [19]:
flightPerFilePath = './flight-data/departuredelays.csv'
airportsFilePath = './flight-data/airport-codes-na.txt'

airports = spark.read.csv(airportsFilePath, header='true', inferSchema='true', sep='\t')
airports.createOrReplaceTempView("airports")

In [21]:
# 출발지연 데이터셋 획득
flightPerf = spark.read.csv(flightPerFilePath, header='true')
flightPerf.createOrReplaceTempView("FlightPerformance")

In [22]:
# 출발지연 데이터를 캐시
flightPerf.cache()

DataFrame[date: string, delay: string, distance: string, origin: string, destination: string]

In [23]:
flightPerf.printSchema()

root
 |-- date: string (nullable = true)
 |-- delay: string (nullable = true)
 |-- distance: string (nullable = true)
 |-- origin: string (nullable = true)
 |-- destination: string (nullable = true)



In [24]:
airports.printSchema()

root
 |-- City: string (nullable = true)
 |-- State: string (nullable = true)
 |-- Country: string (nullable = true)
 |-- IATA: string (nullable = true)



### 비행 성능 데이터셋과 공항 데이터셋 조인

In [25]:
spark.sql("""
    select a.City,
    f.origin,
    sum(f.delay) as Delays
    from FlightPerformance f
    join airports a
    on a.IATA = f.origin
    where a.State == 'WA'
    group by a.City, f.origin
    order by sum(f.delay) desc""").show(5)

+-------+------+--------+
|   City|origin|  Delays|
+-------+------+--------+
|Seattle|   SEA|159086.0|
|Spokane|   GEG| 12404.0|
|  Pasco|   PSC|   949.0|
+-------+------+--------+



databricks를 사용해야 함
```python
%sql
# Query Sum of Flight Delays by City and Origin Code (for Washington State)
select a.City, f.origin, sum(f.delay) as Delays
from FlightPerformance f
join airports a
on a.IATA = f.origin
where a.State = 'WA'
group by a.City, f.origin
order by sum(f.delay) desc
```

In [26]:
spark.sql("""select a.City, f.origin, sum(f.delay) as Delays 
          from FlightPerformance f 
          join airports a 
          on a.IATA = f.origin where a.State = 'WA' group by a.City, f.origin order by sum(f.delay) desc""").show()

+-------+------+--------+
|   City|origin|  Delays|
+-------+------+--------+
|Seattle|   SEA|159086.0|
|Spokane|   GEG| 12404.0|
|  Pasco|   PSC|   949.0|
+-------+------+--------+

